# import notebook?

How to execute and import a jupyter notebook?



How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

TODO: not allowed to import cells that are not fully validated and unit tested.


## importNotebook(search)


#### the code



In [1]:
var path = require('path');
var fs = require('fs')

// TODO: combine with id2 from rpc and nicename from notebook export
// TODO: insert niceName and getExports here?
// must have a unique id for each unique cell so that
// individual cells can serve as modules as well as notebooks
// adding the cell id as a part of the filename
function getCellPath(cell) {
    var question = cell.questions && cell.questions[0]
        ? (' aka ' + cell.questions[0].substr(0, 50))
        : ''
    return path.join(path.dirname(path.resolve(cell.filename)), cell.id) + question
}

var tmpResults = {}

// How to test if a notebook has already been imported
function importNotebook(notebook, ctx = {}) {
    var {interpret, makeModule} = require('../Core')
    // accept all arguments as the list of queries
    if(arguments.length > 2) {
        notebook = Array.from(arguments)
        if(typeof arugments[arguments.length - 1] === 'object'
          && !Array.isArray(arugments[arguments.length - 1])) {
            notebook = notebook.slice(0, arguments.length - 1)
            ctx = arugments[arguments.length - 1]
        }
    }
    var package = {}
    if (typeof notebook === 'undefined') {
        return Promise.resolve({})
    }
    
    // TODO: move this sort of thing to cache
    var cells
    if(typeof tmpResults[JSON.stringify(notebook)] !== 'undefined')
        cells = tmpResults[JSON.stringify(notebook)]
    else
        cells = interpret(notebook)
    tmpResults[JSON.stringify(notebook)] = cells
    
    if(typeof cells.code !== 'undefined') {
        console.log(`importing ${notebook} - 1 cell - ${cells.id}`)
        
        return makeModule(cells.source.join(''),
                          getCellPath(cells),
                          ctx)
    }
    
    console.log(`importing ${notebook} - ${cells.length} cells - ${cells.map(c => c.id)}`)

    return cells.reduce((obj, c, i) => {
        var result = makeModule(c.source.join(''),
                                getCellPath(c),
                                ctx)
        
        if(typeof result === 'object') {
            if(typeof result[Object.keys(result)[0]] === 'function') {
                const func = result[Object.keys(result)[0]]
                obj[c.id] = func
                obj[func.name] = func
            }
            Object.assign(obj, result)
        }
        if (typeof result === 'function') {
            obj[c.id] = result
            obj[result.name] = result
        }
        obj[i] = result
        Object.assign(ctx, obj)
        return obj
    }, package)
}

module.exports.importNotebook = importNotebook;
module.exports.import = importNotebook;


## makeModule(code, path [, context])

Imports arbitrary code as a Node module.


#### make a module from a string of code



Import arbitrary code in to the current context?

Import arbitrary code in to a new context?


## options

* useCache: true
* __filename: path or filename in errors
* 


TODO: webpack other languages and import

TODO: use zeromq to import any language using other kernels

TODO: use language server from VS Code


#### the code

how does node module require work?

How to turn jupyter notebooks in to modules?


In [ ]:
var Module = require('module').Module
var path = require('path')

function getCached(filepath) {
    // must have a new name for every generation otherwise cache will be returned
    var mtime = fs.statSync(filepath
    // TODO: find a better way to serve notebook names
        .replace(/\.ipynb(\[[0-9]+\]).*$/ig, '.ipynb')).mtime.getTime();

    // TODO: don't use cache of parent modules that have includes that have changed
    var cachedModule = Module._cache[filepath];
    if (cachedModule && mtime < cachedModule.buildTime) {
        return cachedModule;
    }
}

function makeModule(code, pathToCode, ctxGlobal) {
    var filepath = path.resolve(process.cwd(), pathToCode);
    
    ctxGlobal = ctxGlobal || {}
    ctxGlobal.module = getCached(filepath)
    if(ctxGlobal.module) {
        return ctxGlobal.module.exports
    }
    Object.assign(ctxGlobal, {
        importer: require('../Core'),
        module: new Module(filepath, module),
        exports: {original: true},
        __dirname: path.dirname(pathToCode),
        __filename: pathToCode.split('[')[0],
        $$: void 0
    })
    Object.assign(ctxGlobal.module, {
        module: ctxGlobal.module,
        exports: ctxGlobal.exports,
        parent: module,
        buildTime: (new Date()).getTime(),
        filename: pathToCode,
        paths: Module._nodeModulePaths(path.dirname(filepath))
    })
    Module._cache[filepath] = ctxGlobal.module;
    
    // this is basically the magic of the 'rewire' module, reuse this?
    const validVars = Object.keys(ctxGlobal)
        .filter(k => k.match(/^(\$\$|[a-z_][a-z0-9_]*)$/i))
        .join(',');

    if(!pathToCode.includes('Core')) {
        console.info(`compiling ${pathToCode} with ${validVars}`);
    }
    
    // TODO: use the transpiler or kernels here
    ctxGlobal.module._compile(`
module.exports = (({${validVars}}) => {
${code}
return module.exports || exports || {}
})`, filepath);
    
    // assign the actual imports internally by calling the func export
    var wrapperContext = ctxGlobal.module.exports;
    var result = wrapperContext(ctxGlobal);
    ctxGlobal.module.loaded = true;
    ctxGlobal.module.exports = result;
    return result;
}

module.exports.makeModule = makeModule;


## displayCell(id)


#### the code

How to display interpreted results in markdown?



In [ ]:
function resultMarkdown(res) {
    return ('\n\n\n' + res.length + ' match'
        + (res.length !== 1 ? 'es' : '')
        + ' found: ' + res.join(' , ') + '\n\n\n'
        + (res.length > 0
            ? ('\n\n\n' + res.markdown.join('\n') + '\n\n\n'
                + '```\n\n\n' + res.code + '\n\n\n```\n\n\n')
            : ''));
};

function displayCell(results) {
    return (typeof results[0] !== 'undefined' && typeof results[0] !== 'string'
        ? results.reduce((str, res) => {
            str += resultMarkdown(res);
            return str;
        }, '')
        : resultMarkdown(results));
}

module.exports.displayCell = displayCell


## interpret(query)

How to answer questions using jupyter notebooks and simple search?

How to search jupter notebooks for questions and answers?


#### search notebooks using question marks as the index



This function has 4 modes:

```
search terms
search notebook.ipynb
search notebook.ipynb[0]
search notebook.ipynb[search terms]
```

Passing in a string returns the top match of a single cell, unless the string names a notebook in which case it returns and Array of all the cells in the notebook.
`[0]` returns the index of code cells, markdown cells are accumulated leading up to a code cell.
`[search terms]` searches the markdown within the specified notebook, it's garunteed to return a cell from the notebook or no match error.



#### fuse database setup

Handles stale code.

Determine when to update a notebook. Handles the mtime property on cells.

DONE: depends on import cache to build a tree of dependencies

DONE: doesn't depend on Fuse to be installed anymore, uses Utilities/levenshtein.ipynb as a backup

TODO: compare trees generated by matching import commands with fuse and levenshtein algorithms

TODO: scan files for NPM dependencies automatically, scan for includes to automatically inject in to require.Modules

TODO: create better `levSearch()` function that works on arrays and tokens same as fuse.js


In [ ]:

try {
    var Fuse = require('fuse.js')
} catch (e) {
    if(!e.message.includes('Cannot find module')) {
        throw e
    }
}

var FUSE_CONFIG = {
    caseSensitive: false,
    findAllMatches: true,
    distance: 50,
    threshold: 0.5,
    tokenize: true,
    shouldSort: true,
    keys: ['2.questions'],
    id: '1'
}

var token, fuse

function createDatabase(cache) {
    if(typeof Fuse !== 'undefined') {
        token = new Fuse(cache, FUSE_CONFIG)
        fuse = new Fuse(cache, Object.assign({}, FUSE_CONFIG, {
            tokenize: false
        }))
        return
    }
    
    var {
        importNotebook, getCells
    } = require('../Core')
    var searchFunc = getCells(path.resolve(__dirname, '../Utilities/levenshtein.ipynb'))
        .filter(cell => cell.source.join('').includes('function levSearch'))
        .map(cell => path.join(__dirname, '../Utilities', cell.id))
    var levSearch = importNotebook(searchFunc).levSearch
    
    // TODO: turn this in to `levSearch()` function
    token = ({search: levSearch.bind(null, cache, FUSE_CONFIG)})
    fuse = ({search: levSearch.bind(null, cache, FUSE_CONFIG)})
}

function queryDatabase(search) {
    if(typeof fuse === 'undefined') {
        throw new Error(`database not ready! ${search}`)
    }
    var tokenResults = token.search(search)
    var fuseResults = fuse.search(search)
//    console.log(fuseResults)
    return fuseResults
        .filter(s => tokenResults.includes(s))
        .concat(tokenResults.filter(s => fuseResults.includes(s)))
}

function lookupCell(cacheId, cacheCells) {
    const filename = cacheId.replace(/\.ipynb\[[0-9]+\].*/ig, '.ipynb')
    const fresh = cacheCells(filename)
    const mtime = fs.statSync(filename).mtime.getTime()
    const cell = fresh.filter(cell => cell.id === path.basename(cacheId))[0]
    return cell
}

module.exports.createDatabase = createDatabase
module.exports.queryDatabase = queryDatabase
module.exports.lookupCell = lookupCell


#### the code

How to search notebook questions?


In [ ]:
var path = require('path')
var fs = require('fs')

// only load caching functions to prevent recursion
function initializeCache() {
    var {
        getCells, importNotebook, createDatabase
    } = require('../Core')
    var coreNotebooks = [
        path.resolve(__dirname, './files.ipynb'),
        path.resolve(__dirname, './cache.ipynb'),
        path.resolve(__dirname, '../Utilities/levenshtein.ipynb'),
        path.resolve(__dirname, '../Languages/minimatch.ipynb'),
        path.resolve(__dirname, '../Languages/balanced.ipynb'),
    ]
    var cacheLookup = getCells(path.resolve(__dirname, './cache.ipynb'))
        .filter(cell => cell.source.join('').includes('function cacheCells'))
        .map(cell => path.join(__dirname, cell.id))

    cacheCells = importNotebook(cacheLookup).cacheCells
    var tmpCache = [].concat
        .apply([], coreNotebooks.map(n => cacheCells(n)))
        .map(c => [
            c.mtime,
            path.join(path.dirname(c.filename), c.id),
            c
        ])
    
    createDatabase(tmpCache)

    var {cacheAll} = importNotebook('cache all')
    cacheAll()
    cellCache = importNotebook('cell cache').cellCache

    createDatabase(cellCache)
}

var first = true, cacheCells, cellCache;
function interpret(queries) {
    var {
        filterLocal, searchFiles, searchQueryFiles,
        queryDatabase, lookupCell, importNotebook
    } = require('../Core')
    if(first) {
        first = false
        initializeCache()
    }
    
    const fileMode = typeof queries === 'string' && queries.match(/\.ipynb$/ig)
    let searchResults = [].concat.apply([], (typeof queries === 'string'
        ? [queries]
        : queries).map(query => {
        var search = path.basename(query).split(/[\[\]]/ig)
        var searchResults
        if (query.includes('/')) {
            searchResults = filterLocal(query)
        } else if (search.length === 3) {
            searchResults = [searchQueryFiles(search, cellCache)[0]]
        } else if (search[0].match(/\.ipynb$/ig)) {
            searchResults = searchFiles(search[0], cellCache)
        } else {
            searchResults = [queryDatabase(search[0])[0]]
        }
        if(searchResults.length === 0 || typeof searchResults[0] === 'undefined')
            throw new Error(`Nothing found for ${JSON.stringify(query)
                            .substr(0, 200)}`);
        return searchResults
    }))
    if(typeof searchResults[0] === 'undefined') {
        debugger
    }
    if(typeof searchResults[0].source !== 'undefined') {
        return searchResults
    }
    return typeof queries === 'string' && !fileMode
        ? lookupCell(searchResults[0], cacheCells)
        : searchResults.map(r => lookupCell(r, cacheCells));
}

module.exports.interpret = interpret;


#### types of searches



In [ ]:
var fs = require('fs')

// read notebook from files and make a module
function filterLocal(query) {
    var {getCells} = require('../Core')
    var filename = query.replace(/\.ipynb\[[0-9]+\].*/ig, '.ipynb')
    var query = path.basename(query).split(/[\[\]]/ig);
    if(fs.existsSync(filename)) {
        var local = getCells(filename)
        if(query.length < 3) {
            return local
        } else {
            return [local[parseInt(query[1])]]
        }
    } else {
        return []
    }
}

// filter query results by filename
function filterFilename(arr, fname) {
    return arr
        .filter(id => path.basename(id)
                .substr(0, fname.length + 1) === fname + '[');
}

function searchFiles(query, cellCache) {
    return filterFilename(cellCache.map(c => c[1]),
                          path.basename(query))
        .sort((a, b) => 
              parseInt(a.split(/[\[\]]/ig)[1])
              - parseInt(b.split(/[\[\]]/ig)[1]))
}

function searchQueryFiles(query, cellCache) {
    var isNumeric = parseInt(query[1]) + '' === query[1];
    return isNumeric
        ? [(cellCache.filter(cell => path.basename(cell[1]) === `${query[0]}[${query[1]}]`)[0] || [])[1]]
        : filterFilename(queryDatabase(query[1]), query[0])
}

module.exports.filterLocal = filterLocal
module.exports.filterFilename = filterFilename
module.exports.searchFiles = searchFiles
module.exports.searchQueryFiles = searchQueryFiles


## basic tools



### regexToArray(exp, source[, index])

This should be built in to Javascript probably..


#### the code

How to convert regexp matches to an array?

In [ ]:
function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(i === false ? m : m[i]));
    return co;
}

module.exports.regexToArray = regexToArray;


### runAllPromises(promises[])

Sometimes you want to run promises in series instead of in parallel.


#### the code

How to run all promises sequentially?



In [ ]:
function runAllPromises(promises) {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (typeof func == 'function'
                ? (new Promise(func)) : Promise.resolve(func))
                .then(Array.prototype.concat.bind(result));
        });
    }, Promise.resolve([]));
}

module.exports.runAllPromises = runAllPromises;


### getCells(notebook)



#### the code

How to parse cells from a notebook?



In [ ]:
var path = require('path')

function getCells(notebook, types = ['*', 'code']) {
    notebook = path.resolve(notebook)
    //console.info('reading notebook ' + notebook);
    var json = JSON.parse(fs.readFileSync(notebook))
    var kernel = json.metadata.kernelspec;
    var cells = json.cells
        .filter(c => types.includes(c.cell_type))
        .map((c, i) => Object.assign(c, {
            language: kernel.language,
            filename: notebook,
            id: `${path.basename(notebook)}[${i}]`
        }));
    return cells
}

module.exports.getCells = getCells;
